In [1]:
import os 
os.environ['R_HOME'] = '/Users/idsl/anaconda3/Lib/R'

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

train_data = pd.read_csv('adult.data', header= None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])
test_data = pd.read_csv('adult.test', header= None, skiprows=1,  names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])


t_d = test_data

#編號與特徵無關
train_data = train_data.drop(['fnlwgt'], axis=1)
test_data = test_data.drop(['fnlwgt'], axis=1)

#education與education-num相對應(重複)，故刪除。
train_data.drop(['education'], axis = 1, inplace = True)
test_data.drop(['education'], axis = 1, inplace = True)

# 每個元素的前後空格去掉
train_data = train_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
test_data = test_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#查看各欄"?"各數
# train_data.apply(lambda x: np.sum(x == "?"))
# test_data.apply(lambda x: np.sum(x == "?"))

#把"?"取代為NaT
train_data.replace("?", pd.NaT, inplace = True)
test_data.replace("?", pd.NaT, inplace = True)


# #將缺失值補齊，名目資料填眾數，數值資料填平均值。
fill_data = {'workclass': train_data['workclass'].mode()[0], 'occupation': train_data['occupation'].mode()[0], 'native-country': train_data['native-country'].mode()[0]}
fill_data_test = {'workclass': test_data['workclass'].mode()[0], 'occupation': test_data['occupation'].mode()[0], 'native-country': test_data['native-country'].mode()[0]}

#缺失值填充
train_data.fillna(fill_data, inplace=True)
test_data.fillna(fill_data_test, inplace=True)

#將income欄位>50K改為1，<=50K改為0
train_data['income'] = train_data['income'].apply(lambda x: 0 if x == "<=50K" else 1)
test_data['income'] = test_data['income'].apply(lambda x: 0 if x == '<=50K.' else 1)

from sklearn.preprocessing import MinMaxScaler
# 提取出數值特徵欄位
numerical_columns = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

scaler = MinMaxScaler()

#提取個別特徵欄位之數值，並計算該欄位之max與min，並把該欄位依照該max、min值重新scale至[0,1]
train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data[numerical_columns]), columns=numerical_columns)
train_data[numerical_columns] = train_data_scaled

#提取個別特徵欄位之數值，並計算該欄位之max與min，並把該欄位依照該max、min值重新scale至[0,1]
test_data_scaled = pd.DataFrame(scaler.transform(test_data[numerical_columns]), columns=numerical_columns)
test_data[numerical_columns] = test_data_scaled



train_data = pd.get_dummies(train_data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], dtype=int)
test_data = pd.get_dummies(test_data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'], dtype=int)
test_data['native-country_Holand-Netherlands'] = 0

#刪除重複列
train_data.drop_duplicates(inplace=True)
test_data.drop_duplicates(inplace=True)
#將資料隨機化
from sklearn.utils import shuffle
train_data = shuffle(train_data)



feature_mapping = {
    'native-country_Trinadad&Tobago': 'native-country_Trinadad_and_Tobago',
    'native-country_Outlying-US(Guam-USVI-etc)': 'native-country_Outlying-US-Guam-USVI-etc'
}

train_data.rename(columns=feature_mapping, inplace=True)
test_data.rename(columns=feature_mapping, inplace=True)

train_data = train_data.sort_index(axis=1)
test_data = test_data.sort_index(axis=1)
train_data
# columns = train_data.columns
# with open("columns.csv", "w") as f:
#     f.write("\n".join(columns))

C:\Users\idsl\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,age,capital-gain,capital-loss,education-num,hours-per-week,income,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,...,sex_Female,sex_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
7601,0.452055,0.00000,0.0,0.600000,0.397959,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1646,0.219178,0.00000,0.0,0.600000,0.397959,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
8613,0.424658,0.04386,0.0,0.533333,0.397959,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
788,0.575342,0.00000,0.0,0.533333,0.153061,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
13746,0.082192,0.00000,0.0,0.600000,0.602041,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,0.082192,0.00000,0.0,0.533333,0.346939,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
27630,0.219178,0.00000,0.0,0.533333,0.500000,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
18172,0.493151,0.00000,0.0,0.600000,0.397959,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
14253,0.260274,0.03103,0.0,0.666667,0.397959,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0


In [2]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
robjects.r('''install.packages('https://cran.ma.imperial.ac.uk/bin/windows/contrib/3.6/stringr_1.4.0.zip', repos=NULL, type='source')''')

R[write to console]: trying URL 'https://cran.ma.imperial.ac.uk/bin/windows/contrib/3.6/stringr_1.4.0.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 216784 bytes (211 KB)

R[write to console]: downloaded 211 KB




In [3]:
utils = importr('utils')
C50 = importr("C50")
partykit = importr("partykit")


# graph = robjects.packages.importr("graph")


robjects.r('Sys.setlocale("LC_ALL", "en_US.UTF-8")')

robjects.r('library(C50)')

pandas2ri.activate()

r_train_data = pandas2ri.py2rpy(train_data)
r_test_data = pandas2ri.py2rpy(test_data)


# with (robjects.default_converter + pandas2ri.converter).context():
#   r_train_data = robjects.conversion.get_conversion().py2rpy(train_data)

# with (robjects.default_converter + pandas2ri.converter).context():
#   r_test_data = robjects.conversion.get_conversion().py2rpy(test_data)


robjects.r.assign("r_train_data", r_train_data)
robjects.r.assign("r_test_data", r_test_data)

# 在 R 中建立 C4.5 決策樹模型
# robjects.r('print(r_test_data$"income")')
robjects.r('col<-r_test_data$"income"')




R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In Sys.setlocale("LC_ALL", "en_US.UTF-8") :
R[write to console]: 
 
R[write to console]:  OS reports request to set locale to "en_US.UTF-8" cannot be honored



In [ ]:
# robjects.r('formula <- income ~ age + capital-gain + capital-loss + education-num + hours-per-week + marital-status_Divorced + marital-status_Married-AF-spouse + marital-status_Married-civ-spouse + marital-status_Married-spouse-absent + marital-status_Never-married + marital-status_Separated + marital-status_Widowed + native-country_Cambodia + native-country_Canada + native-country_China + native-country_Columbia + native-country_Cuba + native-country_Dominican-Republic + native-country_Ecuador + native-country_El-Salvador + native-country_England + native-country_France + native-country_Germany + native-country_Greece + native-country_Guatemala + native-country_Haiti + native-country_Holand-Netherlands + native-country_Honduras + native-country_Hong + native-country_Hungary + native-country_India + native-country_Iran + native-country_Ireland + native-country_Italy + native-country_Jamaica + native-country_Japan + native-country_Laos + native-country_Mexico + native-country_Nicaragua + native-country_Outlying-US(Guam-USVI-etc) + native-country_Peru + native-country_Philippines + native-country_Poland + native-country_Portugal + native-country_Puerto-Rico + native-country_Scotland + native-country_South + native-country_Taiwan + native-country_Thailand + native-country_Trinadad&Tobago + native-country_United-States + native-country_Vietnam + native-country_Yugoslavia + occupation_Adm-clerical + occupation_Armed-Forces + occupation_Craft-repair + occupation_Exec-managerial + occupation_Farming-fishing + occupation_Handlers-cleaners + occupation_Machine-op-inspct + occupation_Other-service + occupation_Priv-house-serv + occupation_Prof-specialty + occupation_Protective-serv + occupation_Sales + occupation_Tech-support + occupation_Transport-moving + race_Amer-Indian-Eskimo + race_Asian-Pac-Islander + race_Black + race_Other + race_White + relationship_Husband + relationship_Not-in-family + relationship_Other-relative + relationship_Own-child + relationship_Unmarried + relationship_Wife + sex_Female + sex_Male + workclass_Federal-gov + workclass_Local-gov + workclass_Never-worked + workclass_Private + workclass_Self-emp-inc + workclass_Self-emp-not-inc + workclass_State-gov + workclass_Without-pay')

robjects.r('colnames(r_train_data)[colnames(r_train_data) == "income"] <- "class"')
robjects.r('r_train_data$class <- as.factor(r_train_data$class)')
robjects.r('subset_data <- r_train_data[, !colnames(r_train_data) %in% "class"]')

robjects.r('model <- C5.0(x = subset_data, y = r_train_data$class, trials = 8, control = C5.0Control(CF = FALSE, label = "dick"))')

model = robjects.globalenv['model']



# robjects.r('print(summary(model))')

In [ ]:
# robjects.r('colnames(r_test_data)[colnames(r_test_data) == "income"] <- "class"')
robjects.r('subset_data_test <- r_test_data[, !colnames(r_test_data) %in% "income"]')
r_predictions = robjects.r('''
    # 將模型套用在測試數據                 
    predictions <- predict(model, newdata = subset_data_test)         
    freq1 <-table(predictions, col)
                           
    accuracy<-sum(diag(freq1)/sum(freq1))
    print(accuracy)
''')

In [ ]:
predictions = robjects.globalenv['predictions']

with (robjects.default_converter + pandas2ri.converter).context():
  pd_from_r_df = robjects.conversion.get_conversion().rpy2py(predictions)

from openpyxl import Workbook
#產出Excel(Test data)
wb = Workbook()
ws = wb.active
ws.append(['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income','Predict result'])

for i in range(len(pd_from_r_df)):
    if pd_from_r_df[i] == '0':
        result = '<=50K.'
    else:
        result = '>50K.'
    #將現在loop到原始資料的列轉為list
    li = t_d.iloc[i,:].tolist()
    #
    li.append(result)
    ws.append(li)
wb.save('coolData.xlsx')
# predictions = np.array(predictions)

print(pd_from_r_df)